In [ ]:
# hyperparameters
batch_size = 512 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
n_hidden = 512
n_hidden_l = 4
head_size = 64
max_iters = 10000
eval_interval = 300
learning_rate = 3e-4
device = 'cuda'
eval_iters = 200

In [ ]:
from funcs import Tokenizer
tokenizer = Tokenizer("./input.txt")
tokenizer.encode("hello")

In [ ]:
from model import *

In [ ]:
model = SimpleLanguageModel(tokenizer.vocab_size, n_hidden, n_hidden_l, block_size, head_size, device)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

In [ ]:
context = torch.tensor([tokenizer.encode("h")], dtype=torch.long, device=device)
print(tokenizer.decode(m.generate(context, max_new_tokens=block_size)[0].tolist()))

In [ ]:
# Train and test splits
data = torch.tensor(tokenizer.encode(tokenizer.text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(tokenizer.decode(m.generate(context, max_new_tokens=block_size)[0].tolist()))

In [ ]:
context = torch.tensor([tokenizer.encode("h")], dtype=torch.long, device=device)
print(tokenizer.decode(m.generate(context, max_new_tokens=block_size)[0].tolist()))